In [9]:
#!pip install ultralytics

In [10]:
import ultralytics
ultralytics.__version__

'8.3.50'

In [11]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import os 
import time

model=YOLO('yolov8n.pt')

In [12]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [13]:
tracker=Tracker()
count=0

In [14]:
cap=cv2.VideoCapture('highway.mp4')

In [15]:

down = {}
up = {}
counter_down = []
counter_up = []

red_line_y = 198
blue_line_y = 268
offset = 6


if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []
    
    vehicle_classes = {}

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if c in ['car', 'truck', 'motorcycle', 'bus']:
            list.append([x1, y1, x2, y2])
            vehicle_classes[(x1, y1, x2, y2)] = c
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        obj_class = vehicle_classes.get((x3, y3, x4, y4), "unknown") 
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   
        if id in down:
          
           if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
             elapsed_time=time.time() - down[id] 
             if counter_down.count(id)==0:
                counter_down.append(id)
                distance = 10 # meters 
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6 
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  
                cv2.putText(frame,obj_class,(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
             
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:

           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
             elapsed1_time=time.time() - up[id]
              
             if counter_up.count(id)==0:
                counter_up.append(id)      
                distance1 = 10 
                a_speed_ms1 = distance1 / elapsed1_time
                a_speed_kh1 = a_speed_ms1 * 3.6
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2) 
                cv2.putText(frame,obj_class,(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    
    text_color = (0, 0, 0)  
    yellow_color = (0, 255, 255)  
    red_color = (0, 0, 255)  
    blue_color = (255, 0, 0)  

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 6 cars, 1 truck, 66.4ms
Speed: 4.3ms preprocess, 66.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 1 truck, 85.9ms
Speed: 2.0ms preprocess, 85.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 1 truck, 61.3ms
Speed: 3.0ms preprocess, 61.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 1 truck, 68.2ms
Speed: 3.0ms preprocess, 68.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 truck, 68.5ms
Speed: 2.0ms preprocess, 68.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 1 truck, 60.0ms
Speed: 3.0ms preprocess, 60.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 2 trucks, 61.5ms
Speed: 2.8ms preprocess, 61.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 1 truck, 58.4ms
Speed: 3.0ms preprocess, 58.4ms 


0: 320x640 10 cars, 1 truck, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 49.0ms
Speed: 3.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 53.1ms
Speed: 3.0ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 51.0ms
Speed: 1.0ms preprocess, 51.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 trucks, 59.5ms
Speed: 2.0ms preprocess, 59.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 57.9ms
Speed: 2.0ms preprocess,


0: 320x640 8 cars, 2 trucks, 50.0ms
Speed: 2.0ms preprocess, 50.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 49.6ms
Speed: 1.0ms preprocess, 49.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 48.1ms
Speed: 2.0ms preprocess, 48.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 2 trucks, 48.0ms
Speed: 2.0ms preprocess,

Speed: 1.6ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 cars, 1 bus, 4 trucks, 48.1ms
Speed: 2.0ms preprocess, 48.1ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 cars, 1 bus, 3 trucks, 61.7ms
Speed: 3.0ms preprocess, 61.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 1 bus, 3 trucks, 65.7ms
Speed: 2.0ms preprocess, 65.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 buss, 3 trucks, 64.5ms
Speed: 2.0ms preprocess, 64.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 buss, 3 trucks, 55.1ms
Speed: 2.0ms preprocess, 55.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 buss, 3 trucks, 50.9ms
Speed: 1.0ms preprocess, 50.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 bus, 3 trucks, 50.1ms
Spe


0: 320x640 11 cars, 2 trucks, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 bus, 2 trucks, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 trucks, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 bus, 2 trucks, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 bus, 2 trucks, 47.1ms
Speed: 1.6ms preprocess, 47.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 2 trucks, 48.1ms
Speed: 2.0ms preprocess, 48.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 bus, 2 

...


0: 320x640 13 cars, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 47.8ms
Speed: 2.0ms preprocess, 47.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 12 cars, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 1 truck, 47.8ms
Speed: 2.1ms preprocess, 47.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 15 cars, 1 truck, 48.0ms
Speed: 1.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 cars, 1 truck, 48.7ms
Speed: 2.0ms preprocess, 48.7ms inference, 1.0ms postprocess per image


0: 320x640 9 cars, 1 bus, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 bus, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 51.2ms
Speed: 2.0ms preprocess, 51.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 1 bus, 2 trucks, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 48.0ms
Speed: 2.0ms preprocess, 48.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 2 trucks, 49.0ms
Speed: 1.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 buss, 46.3ms
Speed: 2.0ms preprocess, 46.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 buss, 1 truck, 47.0ms
Speed: 1.0ms preproce


0: 320x640 7 cars, 1 bus, 3 trucks, 53.0ms
Speed: 2.0ms preprocess, 53.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 1 bus, 4 trucks, 51.1ms
Speed: 2.0ms preprocess, 51.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 4 trucks, 69.0ms
Speed: 2.0ms preprocess, 69.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 4 trucks, 61.0ms
Speed: 2.0ms preprocess, 61.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 4 trucks, 58.0ms
Speed: 2.0ms preprocess, 58.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 3 trucks, 68.0ms
Speed: 2.0ms preprocess, 68.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 4 trucks, 53.0ms
Speed: 2.0ms preprocess, 53.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 4 trucks, 66.2ms
Speed: 2.0m


0: 320x640 2 cars, 2 buss, 2 trucks, 66.0ms
Speed: 1.0ms preprocess, 66.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 2 buss, 2 trucks, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 bus, 3 trucks, 58.7ms
Speed: 2.0ms preprocess, 58.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 bus, 2 trucks, 63.0ms
Speed: 2.0ms preprocess, 63.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 bus, 6 trucks, 55.4ms
Speed: 2.0ms preprocess, 55.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 bus, 4 trucks, 53.0ms
Speed: 2.0ms preprocess, 53.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 bus, 4 trucks, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 2.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640

KeyboardInterrupt: 

#  detecting objects and drawing 2 lines on image - red and blue

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import time

model=YOLO('yolov8n.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

tracker=Tracker()
count=0

cap=cv2.VideoCapture('highway_mini.mp4')


while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy() 
    px=pd.DataFrame(a).astype("float")

    list=[]
    vehicle_classes = {}
             
    for index,row in px.iterrows(): 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if c in ['car', 'truck', 'motorcycle', 'bus']:
            list.append([x1, y1, x2, y2])
            vehicle_classes[(x1, y1, x2, y2)] = c
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        obj_class = vehicle_classes.get((x3, y3, x4, y4), "unknown") 
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        red_line_y=198
        blue_line_y=268   
        offset = 7

        cv2.circle(frame,(cx,cy),4,(0,0,255),-1) #draw ceter points of bounding box
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame,obj_class,(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
                 
    

    text_color = (255,255,255)  
    red_color = (0, 0, 255)     
    blue_color = (255, 0, 0) 
    green_color = (0, 255, 0)   

    cv2.line(frame,(172,198),(774,198),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  # seconde line
    cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    cv2.imshow("frames", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

# getting the ids of the cars those are touching red line

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import time
from datetime import datetime

model=YOLO('yolov8n.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

tracker=Tracker()
count=0

cap=cv2.VideoCapture('highway_mini.mp4')

down={}
up={}

counter_down=[]
counter_up=[]

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")

    list=[]
    vehicle_classes = {} 
             
    for index,row in px.iterrows(): 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if c in ['car', 'truck', 'motorcycle', 'bus']:
            list.append([x1, y1, x2, y2])
            vehicle_classes[(x1, y1, x2, y2)] = c
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        obj_class = vehicle_classes.get((x3, y3, x4, y4), "unknown")
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        red_line_y=198
        blue_line_y=268   
        offset = 7

        
           
        ''' condition for red line '''
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
          ''' this if condition is putting the id and the circle on the object when the center of the object touched the red line.'''
          
          down[id]=time.time()  # current time when vehicle touch the red line        
                    
          
          if id in down:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box             
           cv2.putText(frame,obj_class,(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    
    print("Readable time:", formatted_time)
    print(down)  # printing the id and time of the vehicle when it is touching the red line
    

    text_color = (255,255,255)  
    red_color = (0, 0, 255)     
    blue_color = (255, 0, 0)  
    green_color = (0, 255, 0)   

    cv2.line(frame,(172,198),(774,198),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  # seconde line
    cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    cv2.imshow("frames", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

# getting the ids of the cars those are touching red line¶

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import time

model=YOLO('yolov8n.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

tracker=Tracker()
count=0

cap=cv2.VideoCapture('highway_mini.mp4')

down={}
up={}

counter_down=[]
counter_up=[]

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  # added this line
    px=pd.DataFrame(a).astype("float")
   

    list=[]
    vehicle_classes = {}
             
    for index,row in px.iterrows(): 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if c in ['car', 'truck', 'motorcycle', 'bus']:
            list.append([x1, y1, x2, y2])
            vehicle_classes[(x1, y1, x2, y2)] = c
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        obj_class = vehicle_classes.get((x3, y3, x4, y4), "unknown")
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        red_line_y=198
        blue_line_y=268   
        offset = 7      
    

        ''' condition for blue line '''

        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
          up[id]=time.time()   ## current time when vehicle touch the blue line
          if id in up:         
           cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
           cv2.putText(frame,obj_class,(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)        
           counter_up.append(id)        

    print(up) 

    
    text_color = (255,255,255)  
    red_color = (0, 0, 255)     
    blue_color = (255, 0, 0)  
    green_color = (0, 255, 0)    

    cv2.line(frame,(172,198),(774,198),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  # seconde line
    cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    cv2.imshow("frames", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker

model = YOLO('yolov8n.pt')

cap = cv2.VideoCapture('highway.mp4')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

count = 0
tracker = Tracker()
down = {}
up = {}
counter_down = []
counter_up = []

red_line_y = 198
blue_line_y = 268
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []
    vehicle_classes = {}

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if c in ['car', 'truck', 'motorcycle', 'bus']:
            list.append([x1, y1, x2, y2])
            vehicle_classes[(x1, y1, x2, y2)] = c
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        obj_class = vehicle_classes.get((x3, y3, x4, y4), "unknown")
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        


        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   # current time when vehichle touch the first line
        if id in down:
          
           if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
             elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)
             if counter_down.count(id)==0:
                counter_down.append(id)
                distance = 10 # meters - distance between the 2 lines is 10 meters
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6  # this will give kilometers per hour for each vehicle. This is the condition for going downside
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,obj_class,(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
             
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:

           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
             elapsed1_time=time.time() - up[id]
             # formula of speed= distance/time  (distance travelled and elapsed time) Elapsed time is It represents the duration between the starting point and the ending point of the movement.
             if counter_up.count(id)==0:
                counter_up.append(id)      
                distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms1 = distance1 / elapsed1_time
                a_speed_kh1 = a_speed_ms1 * 3.6
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,obj_class,(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    
    text_color = (0, 0, 0)  
    yellow_color = (0, 255, 255)  
    red_color = (0, 0, 255)  
    blue_color = (255, 0, 0)  

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
